In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as sb
import spherpro.db as db

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as gg

import pathlib

%matplotlib inline

In [ ]:
from src.variables import Vars as V
from src.config import Conf as C

In [ ]:
import spherpro.bromodules.plot_condition_images as pci
import spherpro.bromodules.plot_image as plot_image
from src.plots.plot_ad_image import AnndataImagePlotter

## Aim

Show example images of apoptosis for all conditions

## 0) Setup configuration and bro

In [ ]:
C.fn_config = sm.input.fn_config
C.fol_paper = pathlib.Path(sm.output.fol_plots)

In [ ]:
C.conds_plots = ['TNF_GFP-FLAG','TNFRSF1A_GFP-FLAG',
                'GFP_nan']
C.channel_name_parp = 'Yb174'
C.channel_name_casp = 'Yb172'
C.channel_name_gfp = 'Tm169'
C.channels_apoptosis = [C.channel_name_parp, C.channel_name_casp ]
C.default_meas = 'MeanIntensityComp'
C.default_stack = 'FullStackFiltered'
C.default_transform = lambda x: np.log10(x+0.1)


In [ ]:
C.fol_paper.mkdir(exist_ok=True)

In [ ]:
bro = sb.get_bro(C.fn_config)

## 1) Analysis

Plot cleaved Parp/Caspase/Classifier for

In [ ]:
bpci = pci.PlotConditionImages(bro)
ap = AnndataImagePlotter(bro)

Get class

In [ ]:
dat_fil = bro.doquery(bro.session.query(db.object_filters, db.objects.object_number,
                                       db.objects.image_id, db.conditions.condition_name)
                     .join(db.object_filter_names)
                      .join(db.objects)
                      .join(db.images)
                      .join(db.conditions)
                      .filter(db.conditions.condition_name.in_(C.conds_plots))
                     .filter(db.object_filter_names.object_filter_name ==  'modelfitcondflag_v1'))

In [ ]:
dat_fil

In [ ]:
figs = []
for m in [C.channel_name_parp]:
    for c in C.conds_plots:
        figs.append(bpci.plot_hm_conditions(c, m, measurement_name=C.default_meas, transf=C.default_transform))

Select spheres:


In [ ]:
C.exp_img_yap = 1140
C.exp_img_tnf = 1009
C.exp_img_tnfr = 376
C.exp_img_gfp = 161

C.exp_imgs = [
    C.exp_img_tnf,
    C.exp_img_tnfr,
    C.exp_img_gfp
             ]
C.exp_imgs_labels = C.conds_plots

In [ ]:

def plot_examples(chan, cmap=plt.cm.viridis):
    fig, axs = plt.subplots(1,len(C.exp_imgs), figsize=(10,2))
    for i, ax, t in zip(C.exp_imgs, axs.flatten(), C.exp_imgs_labels):
        cond = (bro.session.query(db.conditions.condition_name)
                .join(db.images)
                .filter(db.images.image_id == i).one()[0]
               ).split('_')
        tit = ' '.join(cond)
        bro.plots.heatmask.plt_heatplot([i], stat=C.default_meas,
                                        stack=C.default_stack, channel=chan, ax=ax, colorbar=i == C.exp_imgs[-1],
                                       transform_fkt=C.default_transform, title=tit, censor_max=1,
                                       cmap=cmap, bad_color='k')
        plot_image.add_scalebar(ax)

    plot_image.add_scalebar(axs[0])
    plot_image.adapt_ax_clims(axs.flatten())
    return fig

def plot_examples_raw(chan, cmap='plasma'):
    plane_id = bro.helpers.dbhelp.get_plane_id('FullStackComp', chan)
    
    fig, axs = plt.subplots(1,len(C.exp_imgs), figsize=(10,2))
    for i, ax, t in zip(C.exp_imgs, axs.flatten(), C.exp_imgs_labels):
        cond = (bro.session.query(db.conditions.condition_name)
                .join(db.images)
                .filter(db.images.image_id == i).one()[0]
               ).split('_')
        tit = ' '.join(cond)
        mask = bro.io.masks.get_mask(i, 'cell')

        #bro.plots.heatmask.plt_heatplot([i], stat=C.default_meas,
        #                                stack=C.default_stack, channel=chan, ax=ax, colorbar=i == C.exp_imgs[-1],
        #                               transform_fkt=C.default_transform, title=tit, censor_max=1,
        #                               cmap=plt.cm.viridis, bad_color='k')
        ap._plot_imc(plane_id, i, ax=ax, cmap=cmap, sigma=0)
        ax.set_title(t)
        plot_image.add_scalebar(ax)
        ax.axis('off')
        ap._add_raw_contour(mask, ax=ax)
        
    plt.colorbar(ax.get_images()[0])
    plot_image.add_scalebar(axs[0])
    plot_image.adapt_ax_clims(axs.flatten())
    fig.set_dpi(300)
    return fig

def add_fils(fig, oexp_only=True):
    for ax, imid in zip(fig.axes, C.exp_imgs):
        mask = bro.io.masks.get_mask(imid, 'cell')
        tdat = dat_fil.query(f'{db.objects.image_id.key}=={imid}')
        if oexp_only:
            tdat = tdat.query(f'{db.object_filters.filter_value.key} == 3')
        val = tdat[db.object_filters.filter_value.key]
        objnr = tdat[db.objects.object_number.key]
        ap._add_contour(mask, val, objnr=objnr, ax=ax,
                        cmap=['red','lightyellow', 'blue', 'white'],
                     linestyles='-', linewidths=0.5)
    fig.set_dpi(300)
    return fig

def add_ax_classes(ax, imid):
    mask = bro.io.masks.get_mask(imid, 'cell')
    tdat = dat_fil.query(f'{db.objects.image_id.key}=={imid}')
    val = tdat[db.object_filters.filter_value.key]
    objnr = tdat[db.objects.object_number.key]
    ap._add_contour(mask, val, objnr=objnr, ax=ax,
                    cmap=['red','lightyellow', 'blue', 'white'],
                 linestyles='-', linewidths=0.5)
    return ax

In [ ]:
fig_parp = plot_examples(C.channel_name_parp)

In [ ]:
fig_raw = plot_examples_raw(C.channel_name_parp)


In [ ]:
add_fils(fig_raw)
fig_raw.savefig(C.fol_paper / 'supfig_apoptosis_img_parp.pdf',transparent=True)
fig_raw

In [ ]:
fig_parp = plot_examples(C.channel_name_parp)
fig_parp.savefig(C.fol_paper / 'supfig_apoptosis_parp.pdf',transparent=True)


In [ ]:
plt.show(fig_parp)
add_fils(fig_parp)
fig_parp.savefig(C.fol_paper / 'supfig_apoptosis_parp_classes.pdf',transparent=True)
fig_parp

In [ ]:
fig_casp = plot_examples(C.channel_name_casp)

In [ ]:
fig_casp.savefig(C.fol_paper / 'supfig_apoptosis_casp.pdf',transparent=True)

In [ ]:
add_fils(fig_casp)

In [ ]:
fig_raw = plot_examples_raw('Nd144')
add_fils(fig_raw)

In [ ]:
fig_raw = plot_examples_raw('Sm154')
add_fils(fig_raw)

In [ ]:
fig_raw = plot_examples_raw('Nd143')
add_fils(fig_raw)

In [ ]:
fig_raw = plot_examples('Nd143')
add_fils(fig_raw)

In [ ]:
fig_raw = plot_examples_raw(C.channel_name_gfp)
add_fils(fig_raw)

In [ ]:
f= plot_examples('Dy163')
add_fils(f)

In [ ]:
bpci.plot_hm_conditions(C.conds_plots[0], 'Yb171', measurement_name=C.default_meas, transf=C.default_transform)

In [ ]:
bpci.plot_hm_conditions(C.conds_plots[2], 'Yb171', measurement_name=C.default_meas, transf=C.default_transform)

In [ ]:
h=bro.helpers.anndata.HelpersAnndata(bro)

In [ ]:
def plot_raw_img(imgid, chan, cmap='Greys_r', figsize=(2,2)):
    i = imgid
    plane_id = bro.helpers.dbhelp.get_plane_id('FullStackComp', chan)
    
    fig, ax = plt.subplots(1,1, figsize=figsize)
    cond = (bro.session.query(db.conditions.condition_name)
            .join(db.images)
            .filter(db.images.image_id == i).one()[0]
           ).split('_')
    tit = ' '.join(cond)
    mask = bro.io.masks.get_mask(i, 'cell')

    #bro.plots.heatmask.plt_heatplot([i], stat=C.default_meas,
    #                                stack=C.default_stack, channel=chan, ax=ax, colorbar=i == C.exp_imgs[-1],
    #                               transform_fkt=C.default_transform, title=tit, censor_max=1,
    #                               cmap=plt.cm.viridis, bad_color='k')
    ap._plot_imc(plane_id, i, ax=ax, cmap=cmap, sigma=0)
    #ax.set_title(t)
    plot_image.add_scalebar(ax)
    ax.axis('off')
    ap._add_raw_contour(mask, ax=ax)
    return ax 

In [ ]:
ax = plot_raw_img(C.exp_img_gfp, C.channel_name_gfp)
add_ax_classes(ax, C.exp_img_gfp)
fig = plt.gcf()
fig.set_dpi(400)

In [ ]:
fig_raw = plot_examples_raw(C.channel_name_gfp, cmap='Greys_r')
add_fils(fig_raw, oexp_only=False)


In [ ]:
fig_raw = plot_examples(C.channel_name_gfp, cmap=plt.cm.Greys_r)
add_fils(fig_raw, oexp_only=False)


In [ ]:
def plot_raw_img(imgid, chan, cmap='Greys_r', figsize=(2,2)):
    i = imgid
    plane_id = bro.helpers.dbhelp.get_plane_id('FullStackComp', chan)
    
    fig, ax = plt.subplots(1,1, figsize=figsize)
    cond = (bro.session.query(db.conditions.condition_name)
            .join(db.images)
            .filter(db.images.image_id == i).one()[0]
           ).split('_')
    tit = ' '.join(cond)
    mask = bro.io.masks.get_mask(i, 'cell')

    #bro.plots.heatmask.plt_heatplot([i], stat=C.default_meas,
    #                                stack=C.default_stack, channel=chan, ax=ax, colorbar=i == C.exp_imgs[-1],
    #                               transform_fkt=C.default_transform, title=tit, censor_max=1,
    #                               cmap=plt.cm.viridis, bad_color='k')
    ap._plot_imc(plane_id, i, ax=ax, cmap=cmap, sigma=0)
    #ax.set_title(t)
    plot_image.add_scalebar(ax)
    ax.axis('off')
    ap._add_raw_contour(mask, ax=ax)
    return ax 

In [ ]:
ax = plot_raw_img(C.exp_img_gfp, C.channel_name_gfp)
add_ax_classes(ax, C.exp_img_gfp)
fig = plt.gcf()
fig.set_dpi(400)

In [ ]:
fig_raw = plot_examples_raw(C.channel_name_gfp, cmap='Greys_r')
add_fils(fig_raw, oexp_only=False)


In [ ]:
fig_raw = plot_examples(C.channel_name_gfp, cmap=plt.cm.Greys_r)
add_fils(fig_raw, oexp_only=False)
